### Imports

In [ ]:
!cp ../input/gdcm-conda-install/gdcm.tar .
!tar -xvzf gdcm.tar
!conda install --offline ./gdcm/gdcm-2.8.9-py37h71b2a6d_0.tar.bz2

In [ ]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [ ]:
#2 
import os
import glob
from tqdm import tqdm
from pathlib import Path

import pandas as pd
import numpy as np

# from pydicom.pixel_data_handlers.util import apply_voi_lut
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

from PIL import Image
import gdcm
import pydicom

import cv2
import random

### Data Preparation

In [ ]:
df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
if df.shape[0] == 2477:
    fast_sub = True
    df.to_csv('submission.csv', index=False)
else:
    fast_sub = False

In [ ]:
## set to True for faster inference (uses smaller dataset)
fast_sub = False

In [ ]:
import os

from PIL import Image
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

image_id = []
study_id = []
dim0 = []
dim1 = []
split = 'test'
save_dir = f'/kaggle/tmp/{split}/image/'
os.makedirs(save_dir, exist_ok=True)

for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
    for file in filenames:
        # set keep_ratio=True to have original aspect ratio
        xray = read_xray(os.path.join(dirname, file))
        im = resize(xray, size=512)  
        im.save(os.path.join(save_dir, file.replace('.dcm', '.png')))
        image_id.append(file.replace('.dcm', ''))
        study_id.append(dirname.split('/')[-2])
        dim0.append(xray.shape[0])
        dim1.append(xray.shape[1])
        
        if len(dim0) > 3 and fast_sub:
            break
    if len(dim0) > 3 and fast_sub:
            break
meta = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1, 'study_id': study_id})

In [ ]:
!ls -a "/kaggle/tmp/test/image"

In [ ]:
# !zip -r 'images_set.zip' '/kaggle/tmp/test/image'

In [ ]:
meta.to_csv('test_meta.csv', index=False)

In [ ]:
meta.head()

### Image enhancement using Histogram equalization 

In [ ]:
# def rescale_dcm(dcm_px):
#     zero_one = (dcm_px - dcm_px.min())/((dcm_px.max() - dcm_px.min()))
#     rescaled = (zero_one * 255).astype(np.uint8)
#     return rescaled

# def apply_hist_equalization(array):
#     clahe = cv2.createCLAHE(clipLimit = 2, tileGridSize = (8,8))
#     cl_array = clahe.apply(array)
#     return cl_array

# SAMPLE_MEAN = 134.0
# SAMPLE_STD = 56.0

In [ ]:
# sample_df = meta.head(6)
# sample_df.head(10)

In [ ]:
# sample_df = meta.head(6)
# TRAIN = '/kaggle/input/siim-covid19-detection/test/'

# # Plot
# fig, ax = plt.subplots(nrows = 6, ncols = 4, figsize = (25,30))
# r = 0
# c = 0

# for idx, image in tqdm(sample_df.iterrows(), total = len(sample_df)):

#     dcm_path = glob.glob(os.path.join(TRAIN, 
#                                       image.study_id, 
#                                       "*",
#                                       image.image_id+".dcm"))[0]
# #     print(dcm_path)
    
#     # Read the .dcm metadata
#     dcm = pydicom.dcmread(dcm_path)
      
#     # Get the pixel array
#     dcm_px = dcm.pixel_array

#     if dcm.PhotometricInterpretation == "MONOCHROME1":
#         dcm_px = np.amax(dcm_px) - dcm_px
    
#     # Rescale the values in the 0-255 range
#     dcm_rescaled = rescale_dcm(dcm_px)
    
#     #Histogram equalization
#     cl_array = apply_hist_equalization(dcm_rescaled)
    
# #     std_array = (cl_array - SAMPLE_MEAN)/ SAMPLE_STD
# #     std_array = cv2.resize(std_array, (512,512))
    
    
#     ax[r, c].imshow(dcm_rescaled)

#     ax[r, c+1].hist(dcm_rescaled.flatten(), bins = 100)
#     ax[r, c].set_ylabel(f"image id: {image.image_id}")
    
#     ax[r+1, c].imshow(cl_array)
#     ax[r+1, c].set_ylabel(f"enhanced image id:, {image.image_id}")
#     ax[r+1, c+1].hist(cl_array.flatten(), bins = 100)
    
    
#     c = c + 2
#     if c%4 == 0:
#         c = 0
#         r = r+2


In [ ]:
# fig.savefig("histogram.png")

In [ ]:
# fast_df = pd.read_csv('test_meta.csv')
# fast_df.head()

### Image Level Object Detection

In [ ]:
!mkdir det_txt
!mkdir det
! pip install ../input/siim-libs/addict-2.4.0-py3-none-any.whl >> /dev/null
! pip install ../input/siim-libs/timm-0.4.12-py3-none-any.whl >> /dev/null
! pip install ../input/siim-libs/ensemble_boxes-1.0.6-py3-none-any.whl >> /dev/null
! pip install ../input/siim-libs/loguru-0.5.3-py3-none-any.whl >> /dev/null
! pip install ../input/siim-libs/thop-0.0.31.post2005241907-py3-none-any.whl >> /dev/null
! pip install ../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl >> /dev/null
! pip install ../input/omegaconf/omegaconf-2.0.5-py3-none-any.whl >> /dev/null
! pip install -q ../input/landmark-additional-packages/EfficientNet-PyTorch/EfficientNet-PyTorch-master 
! pip install -U ../input/landmark-additional-packages/timm-0.4.12-py3-none-any.whl # to fix 
! pip install ../input/siim-libs/segmentation_models_pytorch-0.1.3-py3-none-any.whl --no-deps  

#### Yolov5 inference

In [ ]:
!cp -r ../input/siim-v5-mh/* .

In [ ]:
# our inference
# !cp -r ../input/yolov5inference/* .

In [ ]:
!python ./inference.py --is_val 0 --output_path det_txt/ --input_path '/kaggle/tmp/test/image/*png' --weight_path '/kaggle/input/siim-v5-weights/*/*/*/best.pt'

In [ ]:
!cp -r ../input/yolox-inference/* .

In [ ]:
# from kaggle_datasets import KaggleDatasets
# dataset_name = '  ...   '
# GCS_PATH = KaggleDatasets().get_gcs_path(dataset_name)
# print(GCS_PATH)

#### Yolox Inference

In [ ]:
!python inference.py -f ../input/yolox-weights/yolox_weights/yolox_siim_d_f0.py -c ss --path '/kaggle/tmp/test/image/*png' \
    --wei_dir ../input/yolox-weights/yolox_weights/ --conf 0.0001 --nms 0.5 --tsize 384 --save_result --device gpu

#### EfficientDet inference

In [ ]:
!cp -r ../input/siimeffdet/* .

In [ ]:
# from kaggle_datasets import KaggleDatasets
# dataset_name = 'siim-det-models'
# GCS_PATH = KaggleDatasets().get_gcs_path(dataset_name)
# print(GCS_PATH)

In [ ]:
!python predict_oof_det.py --is_val 0 --test_path '/kaggle/tmp/test/image/*png' --model_dir ../input/siim-det-models/

In [ ]:
! cp det/* det_txt

#### Ensemble mechanism

In [ ]:
! python ensemble1.py --input_path  'det_txt/*txt' --image_path '/kaggle/tmp/test/image/*png' --thr 0.001

In [ ]:
#mask_dir = f'/kaggle/tmp/{split}/mask/'
#os.makedirs(mask_dir, exist_ok=True)

In [ ]:
#!cp -r ../input/siimsegs/* .

In [ ]:
#!python train_seg.py --image_path '/kaggle/tmp/test/image/*png' --weight_path best_loss.pth

### Study Level Classification

In [ ]:
!cp -r ../input/siim-cls-code/* .

###### NfNet

In [ ]:
!python main.py -C n_cf11_6 -M test -W ../input/siim-cls-weights/n_cf11_6_f3/
!python main.py -C n_cf11 -M test -W ../input/siim-cls-weights/n_cf11_l1/
!python main.py -C n_cf11_7 -M test -W ../input/siim-cls-weights/n_cf11_7/
!python main.py -C n_cf11_9 -M test -W ../input/siim-cls-weights/n_cf11_9/
!python main.py -C n_cf11_10 -M test -W ../input/siim-cls-weights/n_cf11_10/
!python main.py -C n_cf11_1 -M test -W ../input/siim-cls-weights/n_cf_11_1/
!python main.py -C n_cf11_rot1 -M test -W ../input/siim-cls-weights/n_cf11_rot1/

##### Chexnet

In [ ]:
IMG_SIZE = 512
import tensorflow as tf

base = tf.keras.applications.densenet.DenseNet121(weights=None,
                            include_top=True,
                            input_shape=(IMG_SIZE,IMG_SIZE,3),
                            classes=14
                           )

x = tf.keras.layers.Dense(5,activation='sigmoid')(base.output)
# fine_tune = tf.keras.layers.Dense(NUM_CLASSES,activation='sigmoid')(x)

base = tf.keras.Model(inputs=base.input, outputs= x)

print("CheXNet loaded")
base.trainable= False # freeze most layers
base.training= False

base.layers.pop()


In [ ]:
base.load_weights('../input/model-chexnet/model.h5')

In [ ]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        '/kaggle/tmp/test/',
        classes=['image'],
        target_size=(512, 512),
        color_mode="rgb",
        shuffle = False,
        class_mode=None,
        batch_size=1)

In [ ]:
predictions = base.predict_generator(test_generator)

In [ ]:
import pandas as pd
import os
chexnet_pred = pd.DataFrame(predictions , columns = ['pred_cls1','pred_cls2','pred_cls3','pred_cls4','pred_cls5'])

In [ ]:
files = os.listdir('/kaggle/tmp/test/image')
for i in files:
    files
chexnet_pred['image_id'] = files
chexnet_pred['image_id'] =chexnet_pred['image_id'].apply(lambda x: x.split('.')[0])
chexnet_pred.head(20)

#### EfficientNet

In [ ]:
!pip install /kaggle/input/kerasapplications -q
!pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps

import os

import efficientnet.tfkeras as efn
import tensorflow as tf

def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")

    return strategy


def build_decoder(with_labels=True, target_size=(300, 300), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img

    def decode_with_labels(path, label):
        return decode(path), label

    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img

    def augment_with_labels(img, label):
        return augment(img), label

    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)

    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)

    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)

    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)

    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)

    return dset

strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 16

IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600, 512)

In [ ]:
meta.shape

##### 4 class classification

In [ ]:
sub_df = meta.copy()
test_paths = f'/kaggle/tmp/{split}/image/' + sub_df['image_id'] +'.png'

sub_df['negative'] = 0
sub_df['typical'] = 0
sub_df['indeterminate'] = 0
sub_df['atypical'] = 0


label_cols = sub_df.columns[4:]

test_decoder = build_decoder(with_labels=False, target_size=(IMSIZE[7], IMSIZE[7]), ext='png')
dtest = build_dataset(
    test_paths, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=False, cache=False,
    decode_fn=test_decoder
)


with strategy.scope():
    
    models = []
    
#     models0 = tf.keras.models.load_model(
#         '../input/siim-covid19-efnb7-train-study/model0.h5'
#     )
    
#     models1 = tf.keras.models.load_model(
#         '../input/siim-covid19-efnb7-train-study/model1.h5'
#     )
#     models2 = tf.keras.models.load_model(
#         '../input/siim-covid19-efnb7-train-study/model2.h5'
#     )
#     models3 = tf.keras.models.load_model(
#         '../input/siim-covid19-efnb7-train-study/model3.h5'
#     )
    models4 = tf.keras.models.load_model(
        '../input/siim-covid19-efnb7-train-study/model4.h5'
    )
    
#     models.append(models0)
#     models.append(models1)
#     models.append(models2)
#     models.append(models3)
    models.append(models4)
    
sub_df[label_cols] = sum([model.predict(dtest, verbose=1) for model in models]) / len(models)

In [ ]:
sub_df.to_csv('sheep_df.csv' , index=False)

In [ ]:
sub_df.head()

##### 2 class classification

In [ ]:

sub_df = meta.copy()
test_paths = f'/kaggle/tmp/{split}/image/' + sub_df['image_id'] +'.png'

sub_df['none'] = 0

label_cols = sub_df.columns[4]

test_decoder = build_decoder(with_labels=False, target_size=(IMSIZE[8], IMSIZE[8]), ext='png')
dtest = build_dataset(
    test_paths, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=False, cache=False,
    decode_fn=test_decoder
)

with strategy.scope():
    
    models = []
    
#     models0 = tf.keras.models.load_model(
#         '/kaggle/input/siim-covid19-efnb7-train-fold0-5-2class/model0.h5'
#     )
    
#     models1 = tf.keras.models.load_model(
#         '/kaggle/input/siim-covid19-efnb7-train-fold0-5-2class/model1.h5'
#     )
#     models2 = tf.keras.models.load_model(
#         '/kaggle/input/siim-covid19-efnb7-train-fold0-5-2class/model2.h5'
#     )
#     models3 = tf.keras.models.load_model(
#         '/kaggle/input/siim-covid19-efnb7-train-fold0-5-2class/model3.h5'
#     )
    models4 = tf.keras.models.load_model(
        '/kaggle/input/siim-covid19-efnb7-train-fold0-5-2class/model4.h5'
    )
    
#     models.append(models0)
#     models.append(models1)
#     models.append(models2)
#     models.append(models3)
    models.append(models4)

# weights = {
#     0: 1,
#     1: 1,
#     2: 1,
#     3: 1,
#     4: 3
# }

weights = {
    0: 3,
    1: 1,
    2: 1,
    3: 0,
    4: 3
}
weights_sum = sum(weights.values())
weights = {k: v/weights_sum for k, v in weights.items()}

predictions = [model.predict(dtest, verbose=1) for model in models]
for i, pred in enumerate(predictions):\
    predictions[i] = weights[i] * pred
    
sub_df[label_cols] = sum(predictions)
#sub_df[label_cols] = sum([model.predict(dtest, verbose=1) for model in models]) / len(models)
df_2class = sub_df.reset_index(drop=True)

### Results for classification and detection

In [ ]:
df_2class.head()

In [ ]:
df_2class.to_csv('public_test_sheep_predict_none.csv' , index=False)

In [ ]:
test = pd.read_csv('test_meta.csv')[["image_id", "study_id"]]
test = test.rename(columns={"image_id": "ImageUID", "study_id": "StudyInstanceUID"})
for e in ['Negative for Pneumonia', 'Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance']:
    test[e] = 0

In [ ]:
test.head(25)

In [ ]:
! python tocsv.py --input_path  'test_v5neg_2a.txt' --meta_path 'test_meta.csv'

In [ ]:
def prob2str(row):
    return f'negative {row.pred_cls1:.6f} 0 0 1 1 typical {row.pred_cls2:.6f} 0 0 1 1 indeterminate {row.pred_cls3:.6f} 0 0 1 1 atypical {row.pred_cls4:.6f} 0 0 1 1'
    #return f''

In [ ]:
def combine_image(row):
     return f'none {row.pred_cls5:.6f} 0 0 1 1 {row.PredictionString}'
        #return f''

In [ ]:
def tosub(df1):
    df1_image = df1[['image_id', 'pred_cls1', 'pred_cls5', 'PredictionString']].copy()
    df1_image.rename(columns={"image_id": "id"}, inplace=True)
    df1_image['id'] = df1_image['id'].apply(lambda x: f'{x}_image')
    df1_image['PredictionString'] = df1_image.apply(lambda r: combine_image(r), axis=1) 
    df1_image = df1_image[['id', 'PredictionString']]
    
    df1_study = df1[['study_id', 'pred_cls1', 'pred_cls2', 'pred_cls3', 'pred_cls4']].copy()
    df1_study = df1_study.groupby('study_id').agg('mean').reset_index()
    df1_study.rename(columns={"study_id": "id"}, inplace=True)
    df1_study['id'] = df1_study['id'].apply(lambda x: f'{x}_study')
    df1_study["PredictionString"] = df1_study.apply(lambda r: prob2str(r), axis=1) 
    df1_study = df1_study[['id', 'PredictionString']]

    df1_sub = pd.concat([df1_study, df1_image])

    return df1_sub

In [ ]:
df1 = pd.read_csv('n_cf11_9.csv') 
# df2 = pd.read_csv('n_cf11.csv')#.head(10)
# df3 = pd.read_csv('n_cf11_6.csv')
df4 = pd.read_csv('n_cf11_7.csv')
# df5 = pd.read_csv('n_cf11_10.csv')
# df6 = pd.read_csv('n_cf11_rot1.csv')
# df7 = pd.read_csv('n_cf11_1.csv')
# df8 = pd.read_csv('n_cf11_8.csv')

# df2 = df1[["image_id"]].merge(df2, on=["image_id"])
# df3 = df1[["image_id"]].merge(df3, on=["image_id"])
df4 = df1[["image_id"]].merge(df4, on=["image_id"])
# df5 = df1[["image_id"]].merge(df5, on=["image_id"])
# df6 = df1[["image_id"]].merge(df6, on=["image_id"])
# df7 = df1[["image_id"]].merge(df7, on=["image_id"])
#df8 = df1[["image_id"]].merge(df8, on=["image_id"])

In [ ]:
df1.head()

In [ ]:
chexnet_predictions = chexnet_pred.set_index('image_id')
chexnet_predictions =chexnet_predictions.reindex(index = df1['image_id'])
chexnet_predictions.reset_index()
chexnet_predictions.head()

In [ ]:
sub_df.head()

In [ ]:
sheep_df = pd.read_csv('sheep_df.csv')
sheep_df = sheep_df.rename(columns={"image_id": "image_id", "negative": "pred_cls1", 
                         "typical": "pred_cls2", "indeterminate": "pred_cls3",
                        "atypical": "pred_cls4"})

# sheep_df = df2[["image_id"]].merge(sheep_df, on=["image_id"])

sheep_df = df4[["image_id"]].merge(sheep_df, on=["image_id"])

for col in ['pred_cls1', 'pred_cls2', 'pred_cls3', 'pred_cls4', 'pred_cls5']:
    df1[col] = (df1[col] + df4[col])/2
    
# for col in ['pred_cls1', 'pred_cls2', 'pred_cls3', 'pred_cls4']:
#     df1[col] = (1*df1[col] + sheep_df[col])/2
    
for col in ['pred_cls1', 'pred_cls2', 'pred_cls3', 'pred_cls4']:
    df1[col] = (0.5 *df1[col] + sheep_df[col])/2

In [ ]:
sheep_df.head()

In [ ]:
# df1.head()

In [ ]:
sheep_none_df = pd.read_csv('public_test_sheep_predict_none.csv')
sheep_none_df = df1[["image_id"]].merge(sheep_none_df, on=["image_id"])

#df1['pred_cls5'] = sheep_none_df['Negative for Pneumonia']
df1['pred_cls5'] = 2*(1-df1['pred_cls5']) + 1*sheep_df['pred_cls1'] + 1*sheep_none_df['none']

In [ ]:
image_sub = pd.read_csv('v5_50.csv')
image_sub.head()

In [ ]:
image_sub = pd.read_csv('v5_50.csv')
image_sub = df1[["image_id"]].merge(image_sub, on=["image_id"])
df1['PredictionString'] = image_sub['PredictionString']
df_sub  = tosub(df1)

In [ ]:
df1.head()

In [ ]:
# from IPython.display import FileLink
# FileLink(r'./required.zip')

In [ ]:
image_sub = pd.read_csv('v5_50.csv')
image_sub = df1[["image_id"]].merge(image_sub, on=["image_id"])
df1['PredictionString'] = image_sub['PredictionString']
df_sub  = tosub(df1)

In [ ]:
df_sub.tail()

In [ ]:
df_sub.shape

In [ ]:
df_sub.iloc[:,1]

In [ ]:
!rm -r ./*

In [ ]:
meta.head(20)

In [ ]:
df_sub.to_csv('submission.csv', index=False)